In [2]:
%pip install app_store_scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 138.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 381.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 640.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 490.9 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.

In [38]:
# Membuat class app store scraper
from app_store_scraper import AppStore
import pandas as pd
import os
import time
import random

class AppStoreSraper(object):
    filename = ""

    def __init__(self, app_id: str, app_name: str, country_code: str):
        try:
            self.app_id = app_id
            self.app_name = app_name
            self.country_code = country_code
            self.scraper =  AppStore(country=self.country_code, app_id=self.app_id, app_name=self.app_name)

        except Exception as e:
            print(f'Fail to init :{e}')

    def __get_reviews(self, max_reviews=100) -> list:
        reviews = []
        self.scraper.review(how_many=max_reviews)
        for review in self.scraper.reviews:
            data = {
                'user_name': review.get('userName'),
                'title': review.get('title'),
                'rating': review.get('rating'),
                'review': review.get('review'),
                'date': review.get('date'),
                'response':None
            }
            if review.get('developerResponse') is not None:
                data['response'] = review.get('developerResponse').get("body")
            reviews.append(data)
        return reviews
    
    def __save_reviews_to_csv(self, reviews: list, filename="app_store_reviews.csv"):
        self.filename = filename
        df = pd.DataFrame(reviews)
        if not os.path.exists(filename):
            df.to_csv(filename, index=False)
        else:
            df.to_csv(filename, mode='a', header=False, index=False)

    
    def scrapper_reviews(self, max_reviews = 100, batch_size=20) ->list:
        all_reviews = []
        for _ in range(0, max_reviews, batch_size):
            try:
                reviews = self.__get_reviews(batch_size)

                if self.filename != "":
                    self.__save_reviews_to_csv(reviews, self.filename)
                else:
                    self.__save_reviews_to_csv(reviews)

                all_reviews.extend(reviews)
                print(f'Success get {len(reviews)} reviews')
                waiting_time = random.uniform(5, 10)
                print(f'Waiting {waiting_time} seconds')
                time.sleep(waiting_time)
            except Exception as e:
                print(f'Got error : {e}')
                waiting_time = random.uniform(60, 120)
                print(f'Waiting {waiting_time} seconds')
                time.sleep(waiting_time)
        return all_reviews
    
    def get_total_scrap(self) ->int:
        # open file by pandas
        try:
            df = pd.read_csv(self.filename)
        except FileNotFoundError as e:
            print(f'File not found : {e}')
            return 0
        return len(df)

    def drop_duplicate(self):
        dataset_df = pd.read_csv(self.filename)

        dataset_df = dataset_df.drop_duplicates()

        filename= "amazon_app_store_reviews_remove_duplicate.csv"

        if not os.path.exists(filename):
            dataset_df.to_csv(filename, index=False)
        else:
            dataset_df.to_csv(filename, mode='a', header=False, index=False)
        

In [39]:
# buat objek appstore sraper
scrap = AppStoreSraper(app_id="297606951",app_name="Amazon Shopping",country_code="us")

scrap.filename = "amazon_app_store_reviews.csv"

scrap.scrapper_reviews(max_reviews=10000)

2024-07-20 00:56:16,490 [INFO] Base - Initialised: AppStore('us', 'amazon-shopping', 297606951)
2024-07-20 00:56:16,491 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/amazon-shopping/id297606951
2024-07-20 00:56:16,544 [INFO] Base - [id:297606951] Fetched 20 reviews (20 fetched in total)


Success get 20 reviews
Waiting 6.870961410329853 seconds


2024-07-20 00:56:23,504 [INFO] Base - [id:297606951] Fetched 20 reviews (40 fetched in total)


Success get 40 reviews
Waiting 5.564387130385581 seconds


2024-07-20 00:56:29,146 [INFO] Base - [id:297606951] Fetched 20 reviews (60 fetched in total)


...

2024-07-20 02:24:07,473 [INFO] Base - [id:297606951] Fetched 20 reviews (9980 fetched in total)


Success get 9980 reviews
Waiting 7.913150730189449 seconds


2024-07-20 02:24:15,919 [INFO] Base - [id:297606951] Fetched 20 reviews (10000 fetched in total)


Success get 10000 reviews
Waiting 6.0338159017334725 seconds


[{'user_name': 'ThomasDav',
  'title': 'Absolutely Hate the New UI and Color Crime',
  'rating': 5,
  'review': 'I’m not normally one to complain, but the new app update was a grand waste of time and effort.  If you paid for that abomination, I would demand your money back... and an apology.\nThe user interface change is so different from the previous app that it makes for a very confusing and frustrating experience.  I spent more time figuring out the cluttered navigation than I did shopping - and I apologize, but I do not believe you could have found a more hideous chimera of accent colors if you applied two AI’s and a team of failed design school dropouts to the task.  That sickly, washed-out mint green, tried to be aqua, and God I wish I were blue, is absolutely repugnant and nauseating to behold.  Marketing FAIL!  Please, for the sake of those of us with good color vision make it go away.  I will have to use the mobile website until it is purged.   If you just put it back the way 

In [40]:
scrap.get_total_scrap()

2505000

In [ ]:
scrap.drop_duplicate()